<a href="https://colab.research.google.com/github/kjsampler/Using-AI-to-Predict-Dog-Licenses/blob/main/Multivariate_Prediction_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rather than clutter another notebook, I've decided to show the process of writing the multivariate prediction function in its own seperate notebook. This will allow for more organization and a better ability to explain the process. 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import table
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time
import datetime
from sklearn.metrics import mean_squared_error
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout
import numpy as np
import tensorflow as tf

# Import Data and Format

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
X5_train = np.load("drive/MyDrive/X5_train.npy")
X5_test = np.load("drive/MyDrive/X5_test.npy")

In [29]:
model_5 = tf.keras.models.load_model('drive/MyDrive/model_5')

In [12]:
X_train = np.load("drive/MyDrive/X_train.npy")
X_test = np.load("drive/MyDrive/X_test.npy")

In [21]:
y_train = np.load("drive/MyDrive/y_train.npy")
y_test = np.load("drive/MyDrive/y_test.npy")

# Prepare Output DataFrame

In [5]:
output = pd.DataFrame({'Valid Date': pd.date_range(start = '2017-12-22', end = '2018-12-20', freq = 'B')})
output = output[(output['Valid Date'] != '2017-12-25') &
                (output['Valid Date'] != '2018-01-01') &
                (output['Valid Date'] != '2018-01-15') &
                (output['Valid Date'] != '2018-02-19') &
                (output['Valid Date'] != '2018-05-28') &
                (output['Valid Date'] != '2018-07-04') &
                (output['Valid Date'] != '2018-09-03') &
                (output['Valid Date'] != '2018-11-12') &
                (output['Valid Date'] != '2018-11-22')]

Attempt using predict_values function from univariate model

In [7]:
def predict_values(training_data, output):
  ''' A function that takes in training data and uses a tuned LSTM neural 
  network to recursively predict data for a given number of steps.

  Inputs: training_data: The last 5 points of the models training set
    output: user provided dataframe formatted with a desired time period to be predicted
    as a pd.TimeStamp 
  Returns: The output parameter with a scaled prediction and unscaled prediction column 
  appended'''

  predictions = []
  predictions.append(training_data)
  x_array = np.array(predictions)
  #ammend function to intake number of input variables
  x_array = x_array.reshape(1, 5, 1)
#Generate predictions
  model = model_5
  for n in range(len(output)):
    pred = model.predict(x_array)
    x_array = np.append(x_array, pred)
    predictions.append(pred)

    x_array = x_array[-5:].reshape(1, 5, 1)

#Add predictions to output df, unlist the values
  predictions = predictions[1:]
  pred_list = []
  for n in range(len(output)):
    pred_list.append(predictions[n].tolist())

  unlisted = []
  for n in range(len(pred_list)):
    var = pred_list[n]
    var = var[0]
    unlisted.append(var)

  output['Scaled Prediction'] = unlisted

#Unscale predicted value
  unlisted_array = np.array(unlisted)
  unlisted_array = unlisted_array.reshape(-1, 1)
  output['Predicted Value'] = scaler.inverse_transform(unlisted_array)

  return output

In [8]:
year_one = predict_values(X5_train[-5:], output)

ValueError: ignored

In [9]:
#so the issue is that in a univariate model, i could just take the last 5 data points
#each point was a unique entry
#now, the last 5 points would correlate all to the same variable
#so i need to figure out how to grab the last point from var1, var2 etc

In [10]:
#I just need to use the same data from the last experiment.
#just amend it to 10 features, but not 5 lags.  

In [13]:
X_train.shape

(727, 1, 10)

In [14]:
X_train = X_train.reshape(727,10)

In [15]:
train_df = pd.DataFrame(X_train)
train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.054054,0.014493,0.050360,0.011628,0.008475,0.007812,0.008909,0.013959,0.020513,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.009009,0.009662,0.000000,0.000000,0.000000,0.000000,0.004454,0.001269,0.000000,0.000000
3,0.072072,0.033816,0.007194,0.040698,0.033898,0.031250,0.044543,0.036802,0.025641,0.006579
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
#now we are getting somewhere. now i know how to call the data
#I just need to adjust the function to take in multiple values
#10 vars work together to produce the target
#it would appear that predict actually DOES want 11 inputs.

In [17]:
#X = 10 features
#y = 1 target
#i need to predict on the last 5 rows of train_df
#without a target
#I'm predicting the target

In [18]:
train_df.head()
#there's 10 vars, we need to add the target for the 11th input

,0,1,2,3,4,5,6,7,8,9
0,0.054054,0.014493,0.050360,0.011628,0.008475,0.007812,0.008909,0.013959,0.020513,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.009009,0.009662,0.000000,0.000000,0.000000,0.000000,0.004454,0.001269,0.000000,0.000000
3,0.072072,0.033816,0.007194,0.040698,0.033898,0.031250,0.044543,0.036802,0.025641,0.006579
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
y_train_df = pd.DataFrame(y_train)
train_df['Target'] = y_train_df
train_df.head()

,0,1,2,3,4,5,6,7,8,9,Target
0,0.054054,0.014493,0.050360,0.011628,0.008475,0.007812,0.008909,0.013959,0.020513,0.000000,0.014834
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.009009,0.009662,0.000000,0.000000,0.000000,0.000000,0.004454,0.001269,0.000000,0.000000,0.000362
3,0.072072,0.033816,0.007194,0.040698,0.033898,0.031250,0.044543,0.036802,0.025641,0.006579,0.039074
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
predict_array = train_df[-5:].values
predict_array

array([[0.0990991 , 0.01449275, 0.02158273, 0.05232558, 0.01271186,
        0.05208333, 0.02672606, 0.02411168, 0.02051282, 0.02631579,
        0.02858177],
       [0.14414414, 0.04830918, 0.05035971, 0.06395349, 0.02542373,
        0.04947917, 0.03563474, 0.02918782, 0.03589744, 0.01973684,
        0.07091172],
       [0.00900901, 0.        , 0.        , 0.01162791, 0.        ,
        0.01041667, 0.00890869, 0.00380711, 0.00512821, 0.        ,
        0.00144718],
       [0.11711712, 0.09178744, 0.13669065, 0.09883721, 0.04237288,
        0.0390625 , 0.04231626, 0.03553299, 0.03589744, 0.05263158,
        0.03111433],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [24]:
predict_array.shape

(5, 11)

In [25]:
predict_array = predict_array.reshape(1,5,11)

In [30]:
model = model_5
for n in range(len(output)):
  pred = model.predict(predict_array)

In [31]:
print(pred)

[[0.01934108]]


In [32]:
#okay. I got pred to run, now I just need to reconcile
#what exactly the output is telling me, and how to use it
#changing input shape to (1, 5, 11) gives me 1 pred

In the original function, I set x_array = the last 5 of the training data. Just vanilla targets, no predictive factors involved  
The issue here is that the model will not accept only targets, it needs the 10 predictors (vars) as well  
Meaning that if my model is shaped to only spit out one predicted target, it still needs predictions on the vars as well?  
If I create a new line in the data frame, it will only have a value in "target", not in vars. And in order to predict the next line, I need it to have all 11 columns filled out.  
So how am i going to fill in those vars? My initial thought is to generate a prediction for each target as well. (If I dispense 4 yorkie licenses today, my model needs to predict how many yorkie, plus all other breed, licenses will be dispensed tomorrow, in order to properly make a "target" prediction.  
I am considering starting this function from scratch at this point- I think what worked for the univariate model may not fit with the multivariate model. 

1. explore how model intakes and outputs data
2. decide on shape of "predictions"
3. amend original function from there

In [33]:
X5_train.shape

(724, 5, 11)

In [34]:
predict_array.shape

(1, 5, 11)

In [35]:
exp = X5_train[0]
print(exp.shape)
print(exp)

(5, 11)
[[0.         0.00483092 0.02158273 0.01744186 0.00423729 0.0078125
  0.00890869 0.00507614 0.         0.         0.0068741 ]
 [0.02702703 0.03381643 0.01438849 0.00581395 0.00423729 0.02604167
  0.0155902  0.00761421 0.01025641 0.03289474 0.01591896]
 [0.00900901 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.00036179]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.00900901 0.00483092 0.00719424 0.00581395 0.00423729 0.00260417
  0.00445434 0.00761421 0.00512821 0.00657895 0.00578871]]


In [36]:
len(X5_train)

724

In [37]:
#the model is actually training on 5*11*724 data points-  
#each "point" is an array of 5*11  
#5 lags, 11 features

In [38]:
#explore the output of the function better
#starting with model.predict

In [39]:
#predict_array[0] = 1, meaning I currently only have 1 row in there
print(predict_array.shape)
print(predict_array)

(1, 5, 11)
[[[0.0990991  0.01449275 0.02158273 0.05232558 0.01271186 0.05208333
   0.02672606 0.02411168 0.02051282 0.02631579 0.02858177]
  [0.14414414 0.04830918 0.05035971 0.06395349 0.02542373 0.04947917
   0.03563474 0.02918782 0.03589744 0.01973684 0.07091172]
  [0.00900901 0.         0.         0.01162791 0.         0.01041667
   0.00890869 0.00380711 0.00512821 0.         0.00144718]
  [0.11711712 0.09178744 0.13669065 0.09883721 0.04237288 0.0390625
   0.04231626 0.03553299 0.03589744 0.05263158 0.03111433]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.        ]]]


In [40]:
#no wait... I have a mismatch. I have 5 rows with 11 columns each. 
#The issue is that in X_train (which predict_array was taken from)
#does NOT have 5 lags. Meaning my shape doesn't match the model

In [41]:
#Now I have to decide: 
#Get rid of the lags
#or figure how to accurately pull the data with 5 lags
#eventually both, of course

In [42]:
#compare accuracy with 5 lags vs. 1 lag
#if small, go ahead and run function with one lag
# we don't need to reinvent the wheel here

In [48]:
X_train = X_train.reshape((X_train.shape[0], 1, 10))
X_test = X_test.reshape((X_test.shape[0], 1, 10))
print(X_train.shape)
print(X_test.shape)

(727, 1, 10)
(312, 1, 10)


In [49]:
now = datetime.datetime.now()
model_1 = Sequential()
model_1.add(LSTM(50, input_shape = (X_train.shape[1], X_train.shape[2])))
model_1.add(Dense(50, activation = 'tanh'))
model_1.add(Dropout(0.3))
model_1.add(Dense(35, activation = 'tanh'))
model_1.add(Dropout(0.3))
model_1.add(Dense(20, activation = 'tanh'))
model_1.add(Dropout(0.3))
model_1.add(Dense(1))
model_1.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse'])

history_1 = model_1.fit(X_train, y_train, epochs = 10, batch_size = 20, validation_data = (X_test, y_test),
                        verbose = True, shuffle = False)

Epoch 1/10
37/37 [==============================] - 3s 18ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0085 - val_mse: 0.0085
Epoch 2/10
37/37 [==============================] - 0s 4ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0077 - val_mse: 0.0077
Epoch 3/10
37/37 [==============================] - 0s 4ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.0088 - val_mse: 0.0088
Epoch 4/10
37/37 [==============================] - 0s 5ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 5/10
37/37 [==============================] - 0s 4ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0088 - val_mse: 0.0088
Epoch 6/10
37/37 [==============================] - 0s 5ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0080 - val_mse: 0.0080
Epoch 7/10
37/37 [==============================] - 0s 4ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 8/10
37/37 [==============================] - 0s 5ms/step - loss: 0.0052 - mse: 0.0

In [46]:
scaler = MinMaxScaler()

In [50]:
#67.19
#making predictions 
yhat = model_1.predict(X_test)
X_test = X_test.reshape((X_test.shape[0], 10))
inv_yhat = np.concatenate((yhat, X_test[:, -10:]), axis = 1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, 0]

y_test = y_test.reshape((len(y_test), 1))
inv_y = np.concatenate((y_test, X_test[:, -10:]), axis = 1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]

mse = mean_squared_error(inv_y, inv_yhat)
print('MSE of prediction: ', np.round(mse, 2))

NotFittedError: ignored